# Text modelling
Maja Andrzejczuk

Piotr Bielecki

Krzysztof Wolny

In [225]:
import numpy as np
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import nltk
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import re


In [156]:
data = pd.read_csv("./reflacx_processed.csv")
data.head()

,study,text,stemming_lemmatization,pos,pos_stemming_lemmatization
0,P102R108387,"['support', 'apparatus', 'pneumothorax', 'enla...","['support', 'apparatus', 'pneumothorax', 'enla...","[('support', 'NN'), ('apparatus', 'NN'), ('pne...","[('support', 'NN'), ('apparatus', 'NN'), ('pne..."
1,P102R379837,"['lines', 'tubes', 'satisfactory', 'position',...","['line', 'tube', 'satisfactory', 'position', '...","[('lines', 'NNS'), ('tubes', 'NNS'), ('satisfa...","[('line', 'NN'), ('tube', 'NN'), ('satisfactor..."
2,P102R558314,"['right', 'subclavian', 'catheter', 'noted', '...","['right', 'subclavian', 'catheter', 'note', 't...","[('right', 'JJ'), ('subclavian', 'JJ'), ('cath...","[('right', 'JJ'), ('subclavian', 'JJ'), ('cath..."
3,P102R765317,"['right', 'subclavian', 'vein', 'approach', 'c...","['right', 'subclavian', 'vein', 'approach', 'c...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein..."
4,P102R915878,"['right', 'subclavian', 'vein', 'catheter', 't...","['right', 'subclavian', 'vein', 'catheter', 't...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein..."


Będziemy używac danych po stemmingu oraz lemmatyzacji

# Load data

In [12]:
FILE_PATH = "../KM1/reflacx/"
FILE_NAME = "reflacx-reports-and-eye-tracking-data-for-localization-of-abnormalities-in-chest-x-rays-1.0.0"
MAIN_PATH = "/main_data"

main_data = {}
metadata = []
with zipfile.ZipFile(FILE_PATH+FILE_NAME+".zip") as archive:
    for file in archive.namelist(): 
        if file.startswith(FILE_NAME+MAIN_PATH+"/metadata_phase"):
            metadata.append(pd.read_csv(archive.open(file)))
        elif file.startswith(FILE_NAME+MAIN_PATH):
            id = file.split("/")
            if not id[2] in main_data:
                main_data[id[2]] = []
            main_data[id[2]].append(pd.read_csv(archive.open(file)))

In [128]:
a = np.where(main_data["P224R056523"][0]==True)

val = [str(x) for x in main_data["P224R056523"][0].iloc[a].columns.values ]
for i in range(len(val)):
    if val[i] in ["Atelectasis","Pleural abnormality"]:
        val[i]=""
val

['', '', 'Groundglass opacity', '']

In [135]:
main_data["P159R351453"][0].empty

True

In [ ]:
# P159R351453

In [259]:
study = []
xmin = []
ymin = []
xmax = []
ymax = []
illness = []
for m in main_data:
    study.append(str(m))
    if main_data[m][0].empty: 
        xmin.append([-1])
        ymin.append([-1])
        xmax.append([-1])
        ymax.append([-1])
        illness.append(0)
    else : 
        xmin.append(main_data[m][0]["xmin"].values)
        ymin.append(main_data[m][0]["ymin"].values)
        xmax.append(main_data[m][0]["xmax"].values)
        ymax.append(main_data[m][0]["ymax"].values)
        illness.append(1)
        
labels = pd.DataFrame({"study":study, "xmin": xmin,"ymin": ymin,"xmax": xmax,"ymax": ymax, "illness":illness})

In [184]:
data[data["study"] == "P300R999525"]

,study,text,stemming_lemmatization,pos,pos_stemming_lemmatization
3031,P300R999525,"['right', 'pectoral', 'pacemaker', 'device', '...","['right', 'pectoral', 'pacemaker', 'device', '...","[('right', 'JJ'), ('pectoral', 'JJ'), ('pacema...","[('right', 'JJ'), ('pectoral', 'JJ'), ('pacema..."


In [232]:
df = pd.merge(data, labels, how="left", on="study", sort = False)
df.head()

,study,text,stemming_lemmatization,pos,pos_stemming_lemmatization,xmin,ymin,xmax,ymax,illness
0,P102R108387,"['support', 'apparatus', 'pneumothorax', 'enla...","['support', 'apparatus', 'pneumothorax', 'enla...","[('support', 'NN'), ('apparatus', 'NN'), ('pne...","[('support', 'NN'), ('apparatus', 'NN'), ('pne...",[734],[1204],[2211],[2175],1
1,P102R379837,"['lines', 'tubes', 'satisfactory', 'position',...","['line', 'tube', 'satisfactory', 'position', '...","[('lines', 'NNS'), ('tubes', 'NNS'), ('satisfa...","[('line', 'NN'), ('tube', 'NN'), ('satisfactor...",[831],[1204],[2223],[2310],1
2,P102R558314,"['right', 'subclavian', 'catheter', 'noted', '...","['right', 'subclavian', 'catheter', 'note', 't...","[('right', 'JJ'), ('subclavian', 'JJ'), ('cath...","[('right', 'JJ'), ('subclavian', 'JJ'), ('cath...","[357, 692]","[1224, 1364]","[1006, 2257]","[2242, 2261]",1
3,P102R765317,"['right', 'subclavian', 'vein', 'approach', 'c...","['right', 'subclavian', 'vein', 'approach', 'c...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...","[451, 1469]","[2114, 1101]","[579, 1730]","[2240, 1470]",1
4,P102R915878,"['right', 'subclavian', 'vein', 'catheter', 't...","['right', 'subclavian', 'vein', 'catheter', 't...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...","[('right', 'JJ'), ('subclavian', 'JJ'), ('vein...",[887],[1081],[2248],[2344],1


In [233]:
df = df[["study", "stemming_lemmatization", "illness"]]
df

,study,stemming_lemmatization,illness
0,P102R108387,"['support', 'apparatus', 'pneumothorax', 'enla...",1
1,P102R379837,"['line', 'tube', 'satisfactory', 'position', '...",1
2,P102R558314,"['right', 'subclavian', 'catheter', 'note', 't...",1
3,P102R765317,"['right', 'subclavian', 'vein', 'approach', 'c...",1
4,P102R915878,"['right', 'subclavian', 'vein', 'catheter', 't...",1
...,...,...,...
3027,P300R998041,"['pneumothorax', 'mediastinum', 'normal', 'con...",0
3028,P300R998488,"['visualized', 'portion', 'support', 'apparatu...",1
3029,P300R998581,"['ng', 'tube', 'diaphragm', 'right', 'ij', 'ca...",0
3030,P300R998853,"['cardiac', 'silhouette', 'enlarge', 'mild', '...",1


In [234]:
#tokenization
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [235]:
#count number of occurences of each word
counts = Counter()
for index, row in df.iterrows():
    counts.update(tokenize(row['stemming_lemmatization']))

In [236]:
df.head()

,study,stemming_lemmatization,illness
0,P102R108387,"['support', 'apparatus', 'pneumothorax', 'enla...",1
1,P102R379837,"['line', 'tube', 'satisfactory', 'position', '...",1
2,P102R558314,"['right', 'subclavian', 'catheter', 'note', 't...",1
3,P102R765317,"['right', 'subclavian', 'vein', 'approach', 'c...",1
4,P102R915878,"['right', 'subclavian', 'vein', 'catheter', 't...",1


In [237]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 1045
num_words after: 735


In [239]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [240]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [241]:
df['encoded'] = df['stemming_lemmatization'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df.head()

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


,study,stemming_lemmatization,illness,encoded
0,P102R108387,"['support', 'apparatus', 'pneumothorax', 'enla...",1,"[[2, 3, 4, 5, 4, 6, 4, 7, 4, 8, 4, 9, 4, 10, 4..."
1,P102R379837,"['line', 'tube', 'satisfactory', 'position', '...",1,"[[2, 16, 4, 17, 4, 18, 4, 19, 4, 20, 4, 21, 4,..."
2,P102R558314,"['right', 'subclavian', 'catheter', 'note', 't...",1,"[[2, 32, 4, 36, 4, 37, 4, 38, 4, 39, 4, 40, 4,..."
3,P102R765317,"['right', 'subclavian', 'vein', 'approach', 'c...",1,"[[2, 32, 4, 36, 4, 51, 4, 52, 4, 53, 4, 54, 4,..."
4,P102R915878,"['right', 'subclavian', 'vein', 'catheter', 't...",1,"[[2, 32, 4, 36, 4, 51, 4, 37, 4, 77, 4, 78, 4,..."


In [242]:
#check how balanced the dataset is
Counter(df['illness'])

Counter({1: 2089, 0: 943})

In [243]:
X = list(df['encoded'])
y = list(df['illness'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

# Models

In [244]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [245]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [246]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [247]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

### LSTM with fixed length input

In [248]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [298]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)
train_model(model_fixed, epochs=50, lr=0.01)

train loss 1.558, val loss 1.429, val accuracy 0.684, and val rmse 0.562
train loss 0.659, val loss 0.622, val accuracy 0.684, and val rmse 0.562
train loss 0.576, val loss 0.572, val accuracy 0.684, and val rmse 0.562
train loss 0.526, val loss 0.532, val accuracy 0.712, and val rmse 0.537
train loss 0.503, val loss 0.507, val accuracy 0.761, and val rmse 0.489
train loss 0.482, val loss 0.479, val accuracy 0.774, and val rmse 0.475
train loss 0.438, val loss 0.442, val accuracy 0.792, and val rmse 0.456
train loss 0.388, val loss 0.379, val accuracy 0.827, and val rmse 0.416
train loss 0.350, val loss 0.353, val accuracy 0.865, and val rmse 0.368
train loss 0.327, val loss 0.322, val accuracy 0.880, and val rmse 0.347


In [296]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)
train_model(model_fixed, epochs=50, lr=0.05)

train loss 1.235, val loss 0.646, val accuracy 0.684, and val rmse 0.562
train loss 0.723, val loss 0.510, val accuracy 0.743, and val rmse 0.507
train loss 0.531, val loss 0.509, val accuracy 0.684, and val rmse 0.562
train loss 0.433, val loss 0.434, val accuracy 0.809, and val rmse 0.437
train loss 0.383, val loss 0.430, val accuracy 0.814, and val rmse 0.431
train loss 0.312, val loss 0.359, val accuracy 0.853, and val rmse 0.383
train loss 0.300, val loss 0.341, val accuracy 0.871, and val rmse 0.358
train loss 0.265, val loss 0.320, val accuracy 0.876, and val rmse 0.352
train loss 0.229, val loss 0.315, val accuracy 0.883, and val rmse 0.342
train loss 0.209, val loss 0.313, val accuracy 0.893, and val rmse 0.327


In [297]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)
train_model(model_fixed, epochs=50, lr=0.1)

train loss 0.650, val loss 1.955, val accuracy 0.392, and val rmse 0.780
train loss 0.828, val loss 0.622, val accuracy 0.682, and val rmse 0.564
train loss 0.627, val loss 0.672, val accuracy 0.682, and val rmse 0.564
train loss 0.587, val loss 0.608, val accuracy 0.677, and val rmse 0.568
train loss 0.575, val loss 0.598, val accuracy 0.677, and val rmse 0.568
train loss 0.537, val loss 0.545, val accuracy 0.679, and val rmse 0.567
train loss 0.488, val loss 0.508, val accuracy 0.758, and val rmse 0.492
train loss 0.468, val loss 0.518, val accuracy 0.733, and val rmse 0.517
train loss 0.465, val loss 0.503, val accuracy 0.759, and val rmse 0.490
train loss 0.452, val loss 0.493, val accuracy 0.753, and val rmse 0.497


### LSTM with variable length input

In [256]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [258]:
model = LSTM_variable_input(vocab_size, 50, 50)
train_model(model, epochs=50, lr=0.1)

train loss 0.654, val loss 0.790, val accuracy 0.417, and val rmse 0.764
train loss 0.675, val loss 0.711, val accuracy 0.415, and val rmse 0.765
train loss 0.332, val loss 0.272, val accuracy 0.900, and val rmse 0.317
train loss 0.168, val loss 0.175, val accuracy 0.936, and val rmse 0.253
train loss 0.120, val loss 0.156, val accuracy 0.946, and val rmse 0.233
train loss 0.090, val loss 0.162, val accuracy 0.952, and val rmse 0.219
train loss 0.075, val loss 0.169, val accuracy 0.957, and val rmse 0.207
train loss 0.065, val loss 0.148, val accuracy 0.962, and val rmse 0.195
train loss 0.059, val loss 0.160, val accuracy 0.959, and val rmse 0.203
train loss 0.053, val loss 0.171, val accuracy 0.960, and val rmse 0.199


In [295]:
model = LSTM_variable_input(vocab_size, 50, 50)
train_model(model, epochs=50, lr=0.05)

train loss 0.864, val loss 0.689, val accuracy 0.684, and val rmse 0.562
train loss 0.570, val loss 0.564, val accuracy 0.733, and val rmse 0.517
train loss 0.519, val loss 0.442, val accuracy 0.773, and val rmse 0.477
train loss 0.319, val loss 0.295, val accuracy 0.898, and val rmse 0.320
train loss 0.193, val loss 0.235, val accuracy 0.924, and val rmse 0.275
train loss 0.132, val loss 0.186, val accuracy 0.934, and val rmse 0.257
train loss 0.095, val loss 0.199, val accuracy 0.932, and val rmse 0.260
train loss 0.079, val loss 0.197, val accuracy 0.941, and val rmse 0.244
train loss 0.068, val loss 0.171, val accuracy 0.946, and val rmse 0.233
train loss 0.057, val loss 0.179, val accuracy 0.947, and val rmse 0.230
